# Implementar un MLP con PyTorch para clasificación basado en el dataset de agresividad
# Uso de word embeddings como representación de datos


<img src="figs/fig-diagrama-clasificador2.png" width="900">


### 1. **Representar los datos en el modelo de word embeddings seleccionado**:  
   - #### Generalmente, solo se tokeniza para separar adecuadamente las palabras.
   - #### Sin embargo, dependiendo del modelos de word embeddings algunos preprocesamientos puede mejorar la representación.
   - #### Por ejemplo: 
      - ##### tokenizar y separar correctamente las oraciones y palabras
      - ##### convertir a minúsculas
      - ##### quitar acentos (dependiendo de la fuente de datos con la que se generaros los embeddings)
      - ##### quitar números y puntuación 


### 3. **Convertir los datos a vectores densos: word embeddings**:  
   - #### En el caso de textos corto a nivel de oración, un vector denso por oración. 

### 4. **Separar los datos para entrenamiento, validación y prueba**:  
   - #### Crear los dataset  con la función train_test_split 
   
### 5. **Definir la arquitectura de la red**:  
   - Definir una red de 2 capas, con funciones PReLU en las capas ocultas y una capa de salida

### 6. **Entrenar el modelo**:  
   - Definir los parámetros de las red como: número de épocas, learning_rate, número de neuronas para las capas ocultas, etc.
   
### 7. **Evaluar el modelo**:  
   - Después del entrenamiento, probar la red con las entradas del conjunto de test y evaluar el desempeño con las métricas: Precisión, Recall, F1-score o F1-Measure y Accuracy.
   


# Cargar los datos de entrenamiento

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize
from sklearn.model_selection import train_test_split
import fasttext

# colocar la semilla para la generación de números aleatorios para la reproducibilidad de experimentos

random_state = 42
torch.manual_seed(random_state)
np.random.seed(random_state)

#cargar los datos
dataset = pd.read_json("./data/data_aggressiveness_es.json", lines=True)
#conteo de clases
print("Total de ejemplos de entrenamiento")
print(dataset.klass.value_counts())
# Extracción de los textos en arreglos de numpy
X = dataset['text'].to_numpy()
# Extracción de las etiquetas o clases de entrenamiento
Y = dataset['klass'].to_numpy()



Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Total de ejemplos de entrenamiento
klass
nonaggressive    3655
aggressive       1477
Name: count, dtype: int64


# Cargar el modelo de Word Embeddings y crear los vectores densos

In [2]:
# Cargar el modelo de word embeddings
ft = fasttext.load_model('/Volumes/data/temp/MX.bin')

# Crear los vectores densos para cada texto. Se espera una oración corta

In [3]:
# Extracción de los textos en arreglos de numpy
# Se aplica una función lambda para obtener el vector de cada texto del conjunto de datos
# El resultado es una nueva columna "embedding"
dataset["embedding"] = dataset["text"].map(lambda x: ft.get_sentence_vector(x))
dataset.head()

,id,klass,text,embedding
0,24,aggressive,@USUARIO Lo que tu no respetas es al país HDP,"[-0.047211677, -0.01908352, -0.01999174, 0.024..."
1,1536,nonaggressive,no sé cómo pedirle a la señora que me ayuda a ...,"[-0.033048205, -0.020297991, 0.025773568, -0.0..."
2,2401,nonaggressive,¿Cómo hacer entender en el grupo de la familia...,"[-0.045047347, 0.0062094647, -0.0025499659, 0...."
3,923,nonaggressive,"Jaja, perdón por esta vida loca. Nos vemos pro...","[-0.015040112, 0.008109028, 0.009956434, 0.006..."
4,6075,nonaggressive,Pero que putas hacen esos argentinos hablando ...,"[-0.029231621, -0.015333876, 0.0017929205, 0.0..."


# Crear la matriz de datos para el entrenamiento, validación y prueba del modelo de la red neuronal

In [4]:
# Cada renglón representa un documento codificado en un texto de embeddings
X = np.vstack(dataset['embedding'].to_numpy())
Y = dataset['klass'].to_numpy()

print("Datos:", X.shape) 
print("Etiquetas:", Y.shape) 



Datos: (5132, 300)
Etiquetas: (5132,)


# Codificar las etiquetas

In [5]:
# TODO: Codificar las etiquetas de los datos a una forma categórica numérica: LabelEncoder.

le = LabelEncoder()
# Normalizar las etiquetas a una codificación ordinal para entrada del clasificador
Y_encoded= le.fit_transform(Y)
print("Clases:")
print(le.classes_)
print("Clases codificadas:")
print(le.transform(le.classes_))


Clases:
['aggressive' 'nonaggressive']
Clases codificadas:
[0 1]


# Preparar los conjuntos de datos  para el entrenamiento, validación y prueba

In [6]:
# Dividir el conjunto de datos en conjunto de entrenamiento (80%) y conjunto de pruebas (20%)

X_train, X_test, Y_train, Y_test =  train_test_split(X, Y_encoded, test_size=0.2, stratify=Y_encoded, random_state=42)

# Dividir el conjunto de entrenamiento en:  entrenamiento (90%) y validación (10%)
X_train, X_val, Y_train, Y_val =  train_test_split(X_train, Y_train, test_size=0.1, stratify=Y_train, random_state=42)



# Codificar las clases en forma one-hot 
NUM_CLASSES = 2
Y_train_one_hot = nn.functional.one_hot(torch.from_numpy(Y_train), num_classes=NUM_CLASSES).float()


# Crear minibatches en PyTorch usando DataLoader
def create_minibatches(X, Y, batch_size):
    # Recibe los documentos en X y las etiquetas en Y
    dataset = TensorDataset(X, Y) # Cargar los datos en un dataset de tensores
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    # loader = DataLoader(dataset, batch_size=batch_size)
    return loader


In [10]:
X_train.shape, Y_train.shape, X_val.shape, Y_val.shape, X_test.shape, Y_test.shape

((3694, 300), (3694,), (411, 300), (411,), (1027, 300), (1027,))

In [11]:
Y_train_one_hot[:5],  Y_train_one_hot.shape

(tensor([[0., 1.],
         [1., 0.],
         [0., 1.],
         [0., 1.],
         [1., 0.]]),
 torch.Size([3694, 2]))

# Definición de la arquitectura de la red

In [12]:

# Definir la red neuronal en PyTorch heredando de la clase base de Redes Neuronales: Module
class FF(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        # Definición de capas, funciones de activación e inicialización de pesos
        input_size_h1 = 128
        input_size_h2 = 8 
        self.fc1 = nn.Linear(input_size, input_size_h1)
        self.fc2 = nn.Linear(input_size_h1, input_size_h2)
        self.output = nn.Linear(input_size_h2, output_size)
        self.act1= nn.PReLU()
        self.act2= nn.PReLU()

        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.xavier_uniform_(self.output.weight)

        if self.fc1.bias is not None:
            nn.init.zeros_(self.fc1.bias)
        if self.fc2.bias is not None:
            nn.init.zeros_(self.fc2.bias)        
        if self.output.bias is not None:
            nn.init.zeros_(self.output.bias)        

    
    def forward(self, X):
        # Definición del orden de conexión de las capas y aplición de las funciones de activación
        x = self.fc1(X)
        x = self.act1(x)
        x = self.fc2(x)
        x = self.act2(x)
        x = self.output(x)
        return x

# Entrenamiento de la red

In [13]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Establecer los parámetros de la red

# Parámetros de la red
input_size =  X_train.shape[1]

output_size = 2   # 2 clases

epochs = 10 # variar el número de épocas, para probar que funciona la programación 
                 # solo usar 2 épocas, para entrenamiento total usar por ejemplo 1000 épocas
learning_rate = 0.001 # Generalmente se usan learning rate pequeños (0.001), 

# Se recomiendan tamaños de batch_size potencias de 2: 16, 32, 64, 128, 256
# Entre mayor el número más cantidad de memoria se requiere para el procesamiento
batch_size = 128 # definir el tamaño del lote de procesamiento 


# Convertir los datos de entrenamiento y etiquetas a tensores  de PyTorch

X_train_t = torch.from_numpy(X_train)
X_train_t = X_train_t.to(torch.float32)
Y_train_t = Y_train_one_hot

X_val_t = torch.from_numpy(X_val)
X_val_t = X_val_t.to(torch.float32)


# Crear la red
model = FF(input_size, output_size)

# Definir la función de pérdida
# Entropía Cruzada 
criterion = nn.CrossEntropyLoss() 

# Definir el optimizador
#Parámetros del optimizador: parámetros del modelo y learning rate 
# Adaptive Moment Estimation
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Entrenamiento
print("Iniciando entrenamiento en PyTorch")


for epoch in range(epochs):
# Poner el modelo en modo de entrenamiento
    model.train()  
    lossTotal = 0
    #definir el batch_size
    dataloader = create_minibatches(X_train_t, Y_train_t, batch_size=batch_size)
    for X_tr, y_tr in dataloader:
        # inicializar los gradientes en cero para cada época
        optimizer.zero_grad()
        
        # Propagación hacia adelante
        y_pred = model(X_tr)  #invoca al método forward de la clase MLP
        # Calcular el error MSE
        loss = criterion(y_pred, y_tr)
        #Acumular el error 
        lossTotal += loss.item()
        
        # Propagación hacia atrás: cálculo de los gradientes de los pesos y bias
        loss.backward()
        
        # actualización de los pesos: regla de actualización basado en el gradiente:
        #  
        optimizer.step()
        if np.random.random() < 0.1:
            print(f"Batch Error : {loss.item()}")

    print(f"Época {epoch+1}/{epochs}, Pérdida: {lossTotal/len(dataloader)}")
    
    # Evalúa el modelo con el conjunto de validación
    model.eval()  # Establecer el modo del modelo a "evaluación"
    with torch.no_grad():  # No  calcular gradientes 
        y_pred = model(X_val_t)
        # Obtiene una única clase, la más probable
        y_pred = torch.argmax(y_pred, dim=1)
        print(f"Época {epoch+1}/{epochs}")
        print("P=", precision_score(Y_val, y_pred, average='macro'))
        print("R=", recall_score(Y_val, y_pred, average='macro'))
        print("F1=", f1_score(Y_val, y_pred, average='macro'))
        print("Acc=", accuracy_score(Y_val, y_pred))


Iniciando entrenamiento en PyTorch
Batch Error : 0.6770150065422058
Batch Error : 0.65428626537323
Época 1/10, Pérdida: 0.6450552817048698
Época 1/10
P= 0.3564476885644769
R= 0.5
F1= 0.4161931818181818
Acc= 0.7128953771289538
Batch Error : 0.6243168711662292
Batch Error : 0.6543532013893127
Batch Error : 0.5273615121841431
Batch Error : 0.6213215589523315


/Users/admin/opt/anaconda3/envs/PT/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/admin/opt/anaconda3/envs/PT/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Batch Error : 0.6050225496292114
Época 2/10, Pérdida: 0.5899707829130107
Época 2/10
P= 0.3564476885644769
R= 0.5
F1= 0.4161931818181818
Acc= 0.7128953771289538
Batch Error : 0.6083561778068542
Batch Error : 0.6114820837974548
Batch Error : 0.5589754581451416
Batch Error : 0.5854372978210449
Batch Error : 0.5097629427909851
Época 3/10, Pérdida: 0.5753231521310478
Época 3/10
P= 0.3564476885644769
R= 0.5
F1= 0.4161931818181818
Acc= 0.7128953771289538
Batch Error : 0.6084818243980408
Batch Error : 0.5501173734664917


/Users/admin/opt/anaconda3/envs/PT/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Batch Error : 0.5683637261390686
Época 4/10, Pérdida: 0.5510002128009138
Época 4/10
P= 0.7384085213032581
R= 0.5330161392954242
F1= 0.48830591373943977
Acc= 0.7274939172749392
Batch Error : 0.5141305327415466
Época 5/10, Pérdida: 0.4966999415693612
Época 5/10
P= 0.7203299750439043
R= 0.6378926360849193
F1= 0.651453790238837
Acc= 0.7615571776155717
Batch Error : 0.5271028876304626
Batch Error : 0.4228875935077667
Batch Error : 0.4410017132759094
Batch Error : 0.4043583571910858
Batch Error : 0.478652685880661
Época 6/10, Pérdida: 0.45055794921414605
Época 6/10
P= 0.7420896930200651
R= 0.6633163648984787
F1= 0.6803611280178535
Acc= 0.7761557177615572
Batch Error : 0.4568142294883728
Batch Error : 0.46392756700515747
Época 7/10, Pérdida: 0.42760384904927223
Época 7/10
P= 0.719383346425766
R= 0.6615520333198357
F1= 0.6761441985685206
Acc= 0.7664233576642335
Batch Error : 0.44882920384407043
Batch Error : 0.38600748777389526
Batch Error : 0.38755857944488525
Época 8/10, Pérdida: 0.420287269

### Modo para predicción de datos

In [14]:
# TODO: Transformar el dataset de test con los mismos preprocesamientos y al  espacio de 
# representación vectorial que el modelo entrenado, es decir, al espacio de la matriz TFIDF

# Convertir los datos de prueba a tensores de PyTorch

X_t = torch.from_numpy(X_test)
X_t = X_t.to(torch.float32)

# Desactivar el comportamiento de modo de  entrenamiento: por ejemplo, capas como Dropout
model.eval()  # Establecer el modo del modelo a "evaluación"

with torch.no_grad():  # No  calcular gradientes 
    y_pred_test= model(X_t)

# y_test_pred contiene las predicciones

# Obtener la clase real
y_pred_test = torch.argmax(y_pred_test, dim=1)

print(y_pred_test)


tensor([0, 1, 1,  ..., 0, 1, 0])


### Evaluación

In [15]:
# TODO: Evaluar el modelo con las predicciones obtenidas y las etiquetas esperadas: 
# classification_report y  matriz de confusión (métricas Precisión, Recall, F1-measaure, Accuracy)

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


print(confusion_matrix(Y_test, y_pred_test))
print(classification_report(Y_test, y_pred_test, digits=4, zero_division='warn'))


[[175 121]
 [ 78 653]]
              precision    recall  f1-score   support

           0     0.6917    0.5912    0.6375       296
           1     0.8437    0.8933    0.8678       731

    accuracy                         0.8062      1027
   macro avg     0.7677    0.7423    0.7526      1027
weighted avg     0.7999    0.8062    0.8014      1027



### Evaluación de datos nuevos

In [16]:

x_datos = ["Ese tipo se llevó mis cosas", "ese hijo de se llevo el dinero", "mi app de calendario no sirve"]
# Transformar los datos a vectores densos: word embeddings
x_datos = [ft.get_sentence_vector(texto) for texto in x_datos]

# Apilar los vectores verticalmente para tener un ejemplo.
vectores = np.vstack(x_datos)

x_datos_t = torch.from_numpy(vectores)
x_datos_t = x_datos_t.to(torch.float32)

model.eval()  # Establecer el modo del modelo a "evaluación"
with torch.no_grad():  # No  calcular gradientes 
    y_pred = model(x_datos_t)
    y_pred = torch.argmax(y_pred, dim=1)
    print(le.inverse_transform(y_pred))

['nonaggressive' 'aggressive' 'nonaggressive']


# Ejercicio 1

In [46]:
# TODO: Modificar el número de neuronas de las capas: capa1 y capa2 y evaluar si el modelo mejora o empeora.


# Ejercicio 2

In [ ]:
# TODO: Aplicar una función lamba para que preprocese los textos con los siguientes características:
#   - Tokenizar los datos para separar términos que estuvieran juntos por puntuación o símbolos extraños (sugerencia: usar word_tokenizer)
#   - convertir a minúsculas
#   - eliminar puntuación y símbolos duplicados
# Comprobar si el rendimiento del modelo mejora o empeora.
